# NOTATION

This notebook is copy, train data making step of following notebook.<br>
https://www.kaggle.com/tfukuda675/update-simple-lstm-simple-data<br>

The above notebook is huge, it is hard to run quickly.<br>
So, I picked up the data making step.

<font color="red">If you think this notebook is usefull, please push upvote botton.</font><br>

Thanks!

# Specify library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
#from joypy import joyplot for matplotlib
import tensorflow as tf
from tqdm import tqdm
import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize, StandardScaler
from sklearn.model_selection import train_test_split, GroupKFold, KFold

tqdm.pandas()


# Settings

debug_mode = [true|false] # when this option is true, reduse data size to debug_data_count size.<br>
save_modified_data = True # when this option is true, save output data in this notebook.

In [ ]:
debug_mode = False
debug_data_count = int(1000)
save_modified_data = True

# Read data

In [ ]:
path = '../input/ventilator-pressure-prediction'
train = pd.read_csv(f"{path}/train.csv")
test = pd.read_csv(f"{path}/test.csv")
submission = pd.read_csv(f'{path}/sample_submission.csv')

# Utilitys

In [ ]:
def data_clean(df):
    ## pickup ignore breath id
    ignore_breath_ids = set()
    
    time_step_diff_limit = 0.04
    for k, grp in tqdm(df.groupby("breath_id")):
        
        ## ignore non liner time_step data
        diff_se = grp["time_step"].diff()
        diff_chk = diff_se[diff_se > time_step_diff_limit]
        if len(diff_chk) != 0:
            ignore_breath_ids.add(k)
            
        ## ignor negative pressure data
        m = grp["pressure"].min()
        if m < 0:
            ignore_breath_ids.add(k)
    
    df = df[~df["breath_id"].isin(np.array(list(ignore_breath_ids)))]
    return df

def change_type(df):
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    return df

def add_features(df):
    df['u_in_cumsum'] = df.groupby('breath_id')['u_in'].cumsum()
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df = df.fillna(0)
    return df

def tf_tpu_or_gpu_or_cpu():
    tpu = None
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        return "tpu"

    elif tf.test.is_gpu_available():
        strategy = tf.distribute.get_strategy()
        print('Running on GPU')
        return "gpu"

    else:
        strategy = tf.distribute.get_strategy()
        print('Running on CPU')
        return "cpu"

# reduce data size in debug_mode case

In [ ]:
if debug_mode:
    train = train[:80*debug_data_count]
    test = test[:80*debug_data_count]

# apply utilitys for data

In [ ]:
train = data_clean(train)
train = add_features(train)
train = change_type(train)

test = add_features(test)
test = change_type(test)

# Save modified train/test data

In [ ]:
train.to_csv("./train_mod.csv")
test.to_csv("./test_mod.csv")

# 